In [1]:
import gc
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# 数据导入

In [2]:
def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    '''
    --------------------------------order--------------------------------
    * This file tells us which set (prior, train, test) an order belongs
    * Unique in order_id
    * order_id in train, prior, test has no intersection
    * this is the #order_number order of this user
    '''
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})

    #  order in prior, train, test has no duplicate
    #  order_ids_pri = priors.order_id.unique()
    #  order_ids_trn = train.order_id.unique()
    #  order_ids_tst = orders[orders.eval_set == 'test']['order_id'].unique()
    #  print(set(order_ids_pri).intersection(set(order_ids_trn)))
    #  print(set(order_ids_pri).intersection(set(order_ids_tst)))
    #  print(set(order_ids_trn).intersection(set(order_ids_tst)))

    '''
    --------------------------------product--------------------------------
    * Unique in product_id
    '''
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission
path_data = '../input/'
priors, train, orders, products, aisles, departments, sample_submission = load_data(path_data)

# 时钟函数 + 新建特征函数

In [ ]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))
            
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

# Product Features
各种product有关的feature 主要是product的购买量和重新购买概率等
包括：
#_user_buy_product_times
#_prod_tot_cnts
#_prod_reorder_tot_cnts
#_prod_buy_first_time_total_cnt
#_prod_buy_second_time_total_cnt
#_prod_reorder_prob
#_prod_reorder_ratio
#_prod_reorder_times

In [ ]:
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')

In [ ]:
priors_orders_detail.loc[:,'_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
priors_orders_detail.head()


In [ ]:
agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
                'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
                '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                            '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}
prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)


In [ ]:
prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
prd['_prod_reorder_ratio'] = prd._prod_reorder_tot_cnts / prd._prod_tot_cnts
prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt

In [ ]:
prd.head()

In [ ]:
products.drop('product_name',axis=1, inplace=True)

In [ ]:
priors_orders_detail = priors_orders_detail.merge(products,on = 'product_id')
priors_orders_detail.head()

# Aisle Features
这部分是我自己加的 通道特征---- 表示某一个通道商品的购买次数、购买率等特征：
#_user_buy_aisles_times
#_aisles_tot_cnts
#_aisles_reorder_tot_cnts
#_aisles_buy_first_time_total_cnt
#_aisles_buy_second_time_total_cnt
#_aisles_reorder_prob
#_aisles_reorder_ratio
#_aisles_reorder_times

In [ ]:

priors_orders_detail.loc[:,'_user_buy_aisles_times'] = priors_orders_detail.groupby(['user_id','aisle_id','order_id']).cumcount() + 1
agg_dict_a = {'user_id':{'_aisles_tot_cnts':'count'}, 
                'reordered':{'_aisles_reorder_tot_cnts':'sum'}, 
                '_user_buy_aisles_times': {'_aisles_buy_first_time_total_cnt':lambda x: sum(x==1),
                                            '_aisles_buy_second_time_total_cnt':lambda x: sum(x==2)}}
ais = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['aisle_id'], agg_dict_a)

In [ ]:
ais['_aisles_reorder_prob'] = ais._aisles_buy_second_time_total_cnt / ais._aisles_buy_first_time_total_cnt

In [ ]:
ais['_aisles_reorder_ratio'] = ais._aisles_reorder_tot_cnts / ais._aisles_tot_cnts
ais['_aisles_reorder_times'] = 1 + ais._aisles_reorder_tot_cnts / ais._aisles_buy_first_time_total_cnt

In [ ]:
ais.head()

# Department Features
自己加的 类似之前两个特征

In [ ]:
priors_orders_detail.loc[:,'_user_buy_dep_times'] = priors_orders_detail.groupby(['user_id','department_id','order_id']).cumcount() + 1

agg_dict_a = {'user_id':{'__dep_tot_cnts':'count'}, 
                'reordered':{'_dep_reorder_tot_cnts':'sum'}, 
                '_user_buy_dep_times': {'_dep_buy_first_time_total_cnt':lambda x: sum(x==1),
                                            '_dep_buy_second_time_total_cnt':lambda x: sum(x==2)}}
dep = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['department_id'], agg_dict_a)

In [ ]:
dep['_dep_reorder_prob'] = dep._dep_buy_second_time_total_cnt / dep._dep_buy_first_time_total_cnt

In [ ]:
dep['_dep_reorder_ratio'] = dep._dep_reorder_tot_cnts / dep.__dep_tot_cnts
dep['_dep_reorder_times'] = 1 + dep._dep_reorder_tot_cnts / dep._dep_buy_first_time_total_cnt
dep.head()

# User Time Features
用户和购买时间有关的特征
#_user_total_orders
#_user_sum_days_since_prior_order
#_user_mean_days_since_prior_order
## 这里增加了个特征
#_user_min_days_since_prior_order 
#考虑增加 order_dow / order_hour_of_day 的相关特征

In [ ]:
agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean',
                                        '_user_min_days_since_prior_order': 'min'}}
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)
users.head()

# User order Features

#用户的购买量相关的特征
#_user_total_products
#_user_distinct_products
#_user_reorder_ratio
#_user_maximum_basket
#_user_average_basket
#time_since_last_order
### 增加了一个特征_user_maximum_basket


In [ ]:
# agg_dict_3 = {'reordered':
#                   {'_user_reorder_ratio': 
#                    lambda x: sum(priors_orders_detail.ix[x.index,'reordered']==1)/
#                              sum(priors_orders_detail.ix[x.index,'order_number'] > 1)},
#                   'product_id':{'_user_total_products':'count', 
#                                 '_user_distinct_products': lambda x: x.nunique()},
#                'add_to_cart_order' : {'_user_minimum_basket' : 'min'}}
# us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)

us = pd.concat([
    priors_orders_detail.groupby('user_id')['product_id'].count().rename('_user_total_products'),
    priors_orders_detail.groupby('user_id')['product_id'].nunique().rename('_user_distinct_products'),
    (priors_orders_detail.groupby('user_id')['reordered'].sum() /
        priors_orders_detail[priors_orders_detail['order_number'] > 1].groupby('user_id')['order_number'].count()).rename('_user_reorder_ratio'),
    priors_orders_detail.groupby('user_id')['add_to_cart_order'].max().rename('_user_maximum_basket')
], axis=1).reset_index()

In [ ]:
us.head()

In [ ]:
users = users.merge(us, how='inner')

In [ ]:
users['_user_average_basket'] = users._user_total_products / users._user_total_orders

In [ ]:
us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]

In [ ]:
us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

In [ ]:
users = users.merge(us, how='inner')
users.head()

# User product interaction Feature
#User和特定的product之间的关联特征
#_up_order_count
#_up_first_order_number
#_up_last_order_number
#_up_average_cart_position
#_up_average_order_dow
#_up_order_rate
#_up_order_since_last_order
#_up_order_rate_since_first_order

In [ ]:
agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                                  '_up_first_order_number': 'min', 
                                  '_up_last_order_number':'max'}, 
                  'add_to_cart_order':{'_up_average_cart_position': 'mean'}, 
                 'order_dow' :{'_up_average_order_dow' : 'mean'}}

In [ ]:
data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                          group_columns_list=['user_id', 'product_id'], 
                                                          agg_dict=agg_dict_4)

In [ ]:
data.head()

这里把之前的特征merge进来了

In [32]:
data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')
data = data.merge(products, how ='inner', on ='product_id')
data = data.merge(ais, how='inner', on='aisle_id').merge(dep, how='inner', on='department_id')


In [33]:
# 该商品购买次数 / 总的订单数
# 最近一次购买商品 - 最后一次购买该商品
# 该商品购买次数 / 第一次购买该商品到最后一次购买商品的的订单数
data['_up_order_rate'] = data._up_order_count / data._user_total_orders
data['_up_order_since_last_order'] = data._user_total_orders - data._up_last_order_number
data['_up_order_rate_since_first_order'] = data._up_order_count / (data._user_total_orders - data._up_first_order_number + 1)

# add user_id to train set
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

In [34]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_average_order_dow,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,...,_dep_buy_first_time_total_cnt,_dep_buy_second_time_total_cnt,_dep_reorder_tot_cnts,_dep_reorder_prob,_dep_reorder_ratio,_dep_reorder_times,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
0,1,196,1.400000,2.5,10,1,10,4660,8000,35791,...,1457351,642288,1757892.0,0.440723,0.65346,2.206224,1.000000,0,1.000000,1.0
1,15,196,2.200000,1.4,5,15,22,4660,8000,35791,...,1457351,642288,1757892.0,0.440723,0.65346,2.206224,0.227273,0,0.625000,NaN
2,19,196,6.333333,5.0,3,2,7,4660,8000,35791,...,1457351,642288,1757892.0,0.440723,0.65346,2.206224,0.333333,2,0.375000,NaN
3,21,196,2.000000,3.0,1,10,10,4660,8000,35791,...,1457351,642288,1757892.0,0.440723,0.65346,2.206224,0.030303,23,0.041667,NaN
4,31,196,15.500000,3.0,2,10,17,4660,8000,35791,...,1457351,642288,1757892.0,0.440723,0.65346,2.206224,0.100000,3,0.181818,NaN


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 46 columns):
user_id                              int64
product_id                           uint64
_up_average_cart_position            float64
_up_average_order_dow                float64
_up_order_count                      int64
_up_first_order_number               int16
_up_last_order_number                int16
_prod_buy_second_time_total_cnt      int64
_prod_buy_first_time_total_cnt       int64
_prod_tot_cnts                       int64
_prod_reorder_tot_cnts               float64
_prod_reorder_prob                   float64
_prod_reorder_ratio                  float64
_prod_reorder_times                  float64
_user_mean_days_since_prior_order    float32
_user_sum_days_since_prior_order     float32
_user_min_days_since_prior_order     float32
_user_total_orders                   int16
_user_total_products                 int64
_user_distinct_products              int64
_user_

# User aisle interaction Feature
自己加的 表示User 和特定aisle之间的联系

In [42]:
agg_dict_5 = {'order_number':{'_ua_order_count': 'count', 
                                  '_ua_first_order_number': 'min', 
                                  '_ua_last_order_number':'max'}, 
                 'order_dow' :{'_ua_average_order_dow' : 'mean'}}

In [43]:
ua = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                          group_columns_list=['user_id', 'aisle_id'], 
                                                          agg_dict=agg_dict_5)


add stats features begin ......
add stats features end ......
time lapsing 14.8709998131 s 



In [44]:
data = data.merge(ua, on=['user_id','aisle_id'])

In [45]:
data['ua_order_rate'] = data._ua_order_count / data._user_total_orders
### here is some problems
data['_ua_order_since_last_order'] = data._user_total_orders - data._ua_last_order_number
###
data['_ua_order_rate_since_first_order'] = data._ua_order_count / (data._user_total_orders - data._ua_first_order_number + 1)

In [48]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_average_order_dow,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,...,_up_order_since_last_order,_up_order_rate_since_first_order,reordered,_ua_average_order_dow,_ua_last_order_number,_ua_first_order_number,_ua_order_count,ua_order_rate,_ua_order_since_last_order,_ua_order_rate_since_first_order
0,1,196,1.400000,2.5,10,1,10,4660,8000,35791,...,0,1.000000,1.0,2.384615,10,1,13,1.300000,0,1.3
1,1,46149,3.000000,2.0,3,8,10,1083,1605,8558,...,0,1.000000,1.0,2.384615,10,1,13,1.300000,0,1.3
2,15,196,2.200000,1.4,5,15,22,4660,8000,35791,...,0,0.625000,NaN,1.555556,22,5,9,0.409091,0,0.5
3,15,30292,3.000000,4.0,1,5,5,43,122,186,...,17,0.055556,NaN,1.555556,22,5,9,0.409091,0,0.5
4,15,48142,2.333333,1.0,3,7,21,520,983,3279,...,1,0.187500,NaN,1.555556,22,5,9,0.409091,0,0.5


In [49]:
# ua.drop(['_ua_last_order_number', '_ua_average_order_dow','_ua_first_order_number'], axis = 1, inplace = True)

# User department interaction Feature
User 和 特定department 之间的联系

In [53]:
agg_dict_6 = {'order_number':{'_udp_order_count': 'count', 
                                  '_udp_first_order_number': 'min', 
                                  '_udp_last_order_number':'max'}, 
                 'order_dow' :{'_udp_average_order_dow' : 'mean'}}

In [54]:
# remain code here
udp = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                          group_columns_list=['user_id', 'department_id'], 
                                                          agg_dict=agg_dict_6)
data = data.merge(udp, on=['user_id','department_id'])

add stats features begin ......
add stats features end ......
time lapsing 11.3980000019 s 



In [56]:
data['udp_order_rate'] = data._udp_order_count / data._user_total_orders
### here is some problems
data['_udp_order_since_last_order'] = data._user_total_orders - data._udp_last_order_number
###
data['_udp_order_rate_since_first_order'] = data._udp_order_count / (data._user_total_orders - data._udp_first_order_number + 1)

In [58]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_average_order_dow,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,...,ua_order_rate,_ua_order_since_last_order,_ua_order_rate_since_first_order,_udp_average_order_dow,_udp_last_order_number,_udp_order_count,_udp_first_order_number,udp_order_rate,_udp_order_since_last_order,_udp_order_rate_since_first_order
0,1,196,1.400000,2.5,10,1,10,4660,8000,35791,...,1.300000,0,1.3,2.384615,10,13,1,1.300000,0,1.300000
1,1,46149,3.000000,2.0,3,8,10,1083,1605,8558,...,1.300000,0,1.3,2.384615,10,13,1,1.300000,0,1.300000
2,15,196,2.200000,1.4,5,15,22,4660,8000,35791,...,0.409091,0,0.5,2.000000,22,20,1,0.909091,0,0.909091
3,15,30292,3.000000,4.0,1,5,5,43,122,186,...,0.409091,0,0.5,2.000000,22,20,1,0.909091,0,0.909091
4,15,48142,2.333333,1.0,3,7,21,520,983,3279,...,0.409091,0,0.5,2.000000,22,20,1,0.909091,0,0.909091


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 60 columns):
user_id                              int64
product_id                           uint64
_up_average_cart_position            float64
_up_average_order_dow                float64
_up_order_count                      int64
_up_first_order_number               int16
_up_last_order_number                int16
_prod_buy_second_time_total_cnt      int64
_prod_buy_first_time_total_cnt       int64
_prod_tot_cnts                       int64
_prod_reorder_tot_cnts               float64
_prod_reorder_prob                   float64
_prod_reorder_ratio                  float64
_prod_reorder_times                  float64
_user_mean_days_since_prior_order    float32
_user_sum_days_since_prior_order     float32
_user_min_days_since_prior_order     float32
_user_total_orders                   int16
_user_total_products                 int64
_user_distinct_products              int64
_user_

这里尝试过保存为pkl文件，但是加入很多特征之后，文件太大无法保存
用.h5格式保存会出现奇怪的提示，说不让用fixed type 要用 table保存。暂时还没看懂。
目前暂时用csv保存 这个很慢。

In [59]:
data.drop(['department_id', 'aisle_id'], axis=1, inplace=True)
train = data.loc[data.eval_set == "train",:]
train.loc[:, 'reordered'] = train.reordered.fillna(0)
X_test = data.loc[data.eval_set == "test",:]
train.to_pickle('train.pkl')
X_test.to_pickle('X_test.pkl')

# 特征建立部分到此结束，目前存在的问题：
#当时加特征的时候，没有一个个加特征进行验证，所以目前很多特征都不确定是否有用。加入冗余特征会使得准确率下降吗, 是否需要重新一个个来验证。
#对time的数据处理还不够深 考虑新加关于order_dow（购买日期是星期几） order_hour（购买时间）的feature 以及time和user product的interaction
#但是再加入特征 电脑应该带不动了

# 其他提升空间：
#由于提供了product name。我们可以挖掘一下product name和各种数据之间的联系。
#比如某些单词能给人留下深刻印象的，可能会有更高的reorder rate
#不过这个暂时不知道如何下手